<a href="https://colab.research.google.com/github/tianyuenyt/MO-PCDE_M9_final_assignment/blob/main/BigQuery_Cloud_Cost_Management_Self_Service_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **BigQuery Cloud Cost Managemtn Self Service Tool**
This tool is designed to help Bigquery user track cloud cost trends, identify optimization opportunities, and improve cost efficiency.

## How It Works  
This tool consists of 3 parts to help analyze and optimize BigQuery costs:
1. **Billing Overview** is the starting point for trend analysis, providing the most accurate cost data through project-level reports.
2. **Compute Cost** provides user-level, job-level, and data product (destination table) costs, which are estimated based on [`INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION`](https://cloud.google.com/bigquery/docs/information-schema-jobs-by-organization).
3. **Storage Cost** includes dataset-level and table-level costs, estimated based on [`INFORMATION_SCHEMA.TABLE_STORAGE`](https://cloud.google.com/bigquery/docs/information-schema-table-storage
)

Please follow the script's guidance to ENTER `date range`, `project name`, and `other granular filters` to refine the data

---
📩 Need Help?  
If you encounter **Permission Issues** or have any questions about this tool and analysis, please contact the *Platform Analytics* Team:  
- Tommy: tommy.wu@nytimes.com
- Tian: tian.yue@nytimes.com
---

---
## 💰 Billing Overview
---

In [47]:
#@title **Billing Overview Part I - Load Billing Data**
#@markdown 👈 **Click this Play button** to start.
#@markdown

#@markdown - If no **Date Range** is entered, the query will **automatically load data from the last two quarters**.
#@markdown - You can refine your results by entering a **custom Date Range** below.


from google.colab import auth
import pandas as pd
from google.cloud import bigquery
from datetime import datetime, timedelta

# Authenticate Google Cloud
auth.authenticate_user()

# Specify GCP project ID
project_id ="nyt-platform-analytics-dbt"
client = bigquery.Client(project=project_id)

# Function to get default date range
def get_default_date_range():
    today = datetime.today()
    current_quarter_month = 3 * ((today.month - 1) // 3) + 1
    first_day_of_this_quarter = datetime(today.year, current_quarter_month, 1)

    if first_day_of_this_quarter.month == 1:
        first_day_of_last_quarter = datetime(today.year - 1, 10, 1)
    else:
        first_day_of_last_quarter = datetime(today.year, current_quarter_month - 3, 1)

    yesterday = today - timedelta(days=1)
    return first_day_of_last_quarter.strftime('%Y-%m-%d'), yesterday.strftime('%Y-%m-%d')

# User input for date range
start_date_input = input("Enter start date (YYYY-MM-DD) or press Enter to use default (first day of last quarter): ").strip()
end_date_input = input("Enter end date (YYYY-MM-DD) or press Enter to use default (yesterday): ").strip()

# Assign defaults if needed
default_start, default_end = get_default_date_range()
start_date = start_date_input if start_date_input else default_start
end_date = end_date_input if end_date_input else default_end

# Validate input dates
try:
    start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
    today_dt = datetime.today()

    if end_date_dt < start_date_dt:
        raise ValueError("Error: The end date cannot be earlier than the start date.")
    if end_date_dt > today_dt:
        raise ValueError("Error: The end date cannot be later than today.")
except ValueError as e:
    raise ValueError(f"Invalid date input: {e}")



# Define SQL query
billing_query = f"""
  SELECT
    _pt AS usage_date,
    project_id,
    CASE
      WHEN service_description = 'BigQuery' AND sku_description LIKE '%Storage%' THEN 'Storage'
      WHEN service_description = 'BigQuery' AND sku_description LIKE '%Analysis%' THEN 'Compute'
      WHEN service_description = 'BigQuery Reservation API' THEN 'Compute'
      ELSE 'BigQuery Others'
    END AS label,
    ROUND(SUM(cost), 2) AS cost
  FROM `nyt-platform-analytics-dbt.prod.stg_bq__invoiced_costs`
  WHERE 1=1
    AND _pt BETWEEN '{start_date}' AND '{end_date}'
  GROUP BY ALL
  ORDER BY usage_date, project_id, label
"""

# ANSI Color Code
RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
BOLD = "\033[1m"
RESET = "\033[0m"

# try:
#     df = client.query(billing_query).to_dataframe()
#     print(f"{GREEN}{BOLD} Your BigQuery Billing data from {start_date} to {end_date} is ready!{RESET}")
# except Exception as e:
#     print(f"{RED}{BOLD} Query failed: {e}{RESET}")


try:
    df = client.query(billing_query).to_dataframe()
    print(f"{GREEN}{BOLD} Your BigQuery Billing data from {start_date} to {end_date} is ready!{RESET}")
except Exception as e:
    error_message = str(e).lower()

    if "permission" in error_message or "access denied" in error_message:
        print(f"{RED}{BOLD} Query failed due to a permission issue: {e}{RESET}")
        print(f"{YELLOW}{BOLD} Please contact the Platform Analytics Team for access.{RESET}")
    else:
        print(f"{RED}{BOLD} Query failed: {e}{RESET}")


Enter start date (YYYY-MM-DD) or press Enter to use default (first day of last quarter): 
Enter end date (YYYY-MM-DD) or press Enter to use default (yesterday): 
 Your BigQuery Billing data from 2024-10-01 to 2025-03-19 is ready!


In [50]:
#@title **Billing Overview Part II - Overview of All Project Costs**
#@markdown 👈 **Click this Play button** to start.
#@markdown
#@markdown Use the Filter option in the top-right corner of the table to refine the data.
#@markdown
#@markdown Click **💾 Download as Excel** to save the data locally.


import pandas as pd
import plotly.express as px
import ipywidgets as widgets
import google.colab.data_table

# Define color mapping for labels
color_mapping = {
    "Compute": "#9BB4C1",
    "Storage": "#FFC440",
    "BigQuery Others": "#CFC4B6"
}

# Ensure data is successfully retrieved before processing
if not df.empty:
    df["usage_date"] = pd.to_datetime(df["usage_date"])

    df["year_month"] = df["usage_date"].dt.to_period("M").astype(str)  # Convert to string for Plotly compatibility

    # --- Interactive Stacked Bar Chart: Monthly Cost Breakdown by Label ---
    monthly_cost = df.groupby(["year_month", "label"])["cost"].sum().reset_index()
    monthly_cost["cost_formatted"] = monthly_cost["cost"].apply(lambda x: f"${x:,.2f}")  # Format with $

    fig_bar = px.bar(
        monthly_cost,
        x="year_month",
        y="cost",
        color="label",
        title="Monthly Cost Breakdown by Label",
        labels={"cost": "Total Cost (USD)", "year_month": "Year-Month", "label": "Label"},
        barmode="stack",
        text="cost_formatted",
        hover_data={"label": True, "cost": ":$.2f"},
        color_discrete_map=color_mapping
    )

    fig_bar.update_traces(texttemplate='%{text}', textposition='outside')
    fig_bar.update_layout(
        xaxis=dict(tickangle=-45),
        legend_title="Label",
        hovermode="x unified",
        yaxis_tickprefix="$",
        yaxis_tickformat=","
    )

    fig_bar.show()

    # --- Interactive Line Chart: Day-to-Day Cost Trend by Label ---
    daily_cost = df.groupby(["usage_date", "label"])["cost"].sum().reset_index()
    daily_cost["cost_formatted"] = daily_cost["cost"].apply(lambda x: f"${x:,.2f}")  # Format with $

    fig_line = px.line(
        daily_cost,
        x="usage_date",
        y="cost",
        color="label",
        title="Day-to-Day Cost Trend by Label",
        labels={"cost": "Cost (USD)", "usage_date": "Date", "label": "Label"},
        hover_data={"label": True, "cost": ":$.2f"},
        markers=True,
        color_discrete_map=color_mapping
    )

    fig_line.update_layout(
        xaxis_title="Date",
        yaxis_title="Cost (USD)",
        xaxis=dict(tickangle=-45),
        legend_title="Label",
        hovermode="x unified",
        yaxis_tickprefix="$",
        yaxis_tickformat=","
    )

    fig_line.show()

    project_monthly_cost = df.groupby(["year_month", "project_id", "label"])["cost"].sum().reset_index()
    project_monthly_cost.rename(columns={"cost": "monthly_cost"}, inplace=True)

    project_monthly_cost["monthly_cost"] = project_monthly_cost["monthly_cost"].round(2)

    pivot_table = project_monthly_cost.pivot(index=["project_id", "label"], columns="year_month", values="monthly_cost").fillna(0)

    pivot_table = pivot_table.reset_index()

    pivot_table = pivot_table.loc[:, ~pivot_table.columns.duplicated()]

    valid_months = [col for col in pivot_table.columns if isinstance(col, str) and col.startswith("202")]  # Ensures only YYYY-MM format
    valid_months = sorted(valid_months)

    column_order = ["project_id", "label"] + valid_months

    pivot_table = pivot_table[column_order]

    display(google.colab.data_table.DataTable(pivot_table))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        file_name = f"/content/monthly_billing_data_for_all_projects_from_{start_date}_to_{end_date}.xlsx"

        pivot_table.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)

    display(download_button)

else:
    print("No data available for visualization.")


year_month,project_id,label,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
0,ai-strat-ops,Compute,0.00,0.00,0.00,0.00,0.00,0.00
1,aristo-sadp-dev,Compute,0.00,0.00,0.00,0.00,0.00,0.00
2,aristo-sadp-dev,Storage,0.00,0.00,0.00,0.00,0.00,0.00
3,aristo-sadp-prd,Compute,0.00,0.00,0.01,0.00,0.00,0.75
4,bisque-dbt-bridge-prd,Compute,0.00,0.00,0.00,3.06,54.95,31.54
...,...,...,...,...,...,...,...,...
735,uxf-dev-4766,Compute,0.00,0.00,0.00,0.00,0.00,0.09
736,wc-ga-167813,Storage,13.34,13.52,13.34,13.33,13.43,7.90
737,webstore-analytics-export-prd,BigQuery Others,0.00,0.06,0.15,0.11,0.01,0.00
738,webstore-analytics-export-prd,Compute,0.04,0.09,0.05,0.13,0.01,0.04


Button(description='💾 Download as Excel', style=ButtonStyle())

---
## 📈 Project Billing
---

In [51]:
#@title **Project Billing - Filter Project & Date Range**
#@markdown 👈 **Click this Play button** to start.
#@markdown
#@markdown Now, enter a `Project Name` and `Date Range` to explore the Daily Costs distribution in more detail.
#@markdown - If no Project Name is entered, data for All Projects will be included by default.
#@markdown - If no Date Range is specified, the entire available data range will be used.


import pandas as pd
import plotly.express as px
import google.colab.data_table
from datetime import datetime, timedelta

# --- Ensure start_date and end_date exist ---
try:
    # Check if they exist
    start_date, end_date
except NameError:
    # If not, define them based on last quarter's range
    today = datetime.today()
    current_quarter_month = 3 * ((today.month - 1) // 3) + 1
    first_day_of_this_quarter = datetime(today.year, current_quarter_month, 1)

    if first_day_of_this_quarter.month == 1:
        start_date = datetime(today.year - 1, 10, 1).strftime('%Y-%m-%d')  # Last year's Q4
    else:
        start_date = datetime(today.year, current_quarter_month - 3, 1).strftime('%Y-%m-%d')  # Previous quarter

    end_date = (today - timedelta(days=1)).strftime('%Y-%m-%d')  # Yesterday

project_id_input = input("Enter a Project ID to filter (or press Enter to include all): ").strip()

subset_start_date_input = input(f"Enter subset start date (YYYY-MM-DD) or press Enter to use {start_date}: ").strip()
subset_end_date_input = input(f"Enter subset end date (YYYY-MM-DD) or press Enter to use {end_date}: ").strip()

subset_start_date = subset_start_date_input if subset_start_date_input else start_date
subset_end_date = subset_end_date_input if subset_end_date_input else end_date

# Convert input dates to datetime format for validation
try:
    subset_start_date_dt = pd.to_datetime(subset_start_date)
    subset_end_date_dt = pd.to_datetime(subset_end_date)

    if subset_end_date_dt < subset_start_date_dt:
        raise ValueError("Error: The subset end date cannot be earlier than the subset start date.")
    if subset_end_date_dt > pd.to_datetime(end_date):
        raise ValueError("Error: The subset end date cannot be later than the overall selected range.")

except ValueError as e:
    raise ValueError(f"Invalid date input: {e}")


df_filtered = df.copy()

if project_id_input:
    df_filtered = df_filtered[df_filtered["project_id"] == project_id_input]

df_filtered = df_filtered[(df_filtered["usage_date"] >= subset_start_date) & (df_filtered["usage_date"] <= subset_end_date)]

if not df_filtered.empty:
    df_filtered["usage_date"] = pd.to_datetime(df_filtered["usage_date"]).dt.strftime('%Y-%m-%d')

    daily_cost_filtered = df_filtered.groupby(["usage_date", "label"])["cost"].sum().reset_index()
    daily_cost_filtered["cost"] = daily_cost_filtered["cost"].round(2)

    daily_cost_filtered = daily_cost_filtered.sort_values(by="usage_date", ascending=False)

    # --- Interactive Line Chart: Filtered Day-to-Day Cost Trend ---
    fig_filtered_line = px.line(
        daily_cost_filtered,
        x="usage_date",
        y="cost",
        color="label",
        title=f"Day-to-Day Cost Trend for {project_id_input if project_id_input else 'All Projects'}",
        labels={"cost": "Cost (USD)", "usage_date": "Date", "label": "Label"},
        hover_data={"label": True, "cost": ":$.2f"},
        markers=True,
        color_discrete_map=color_mapping
    )

    fig_filtered_line.update_layout(
        xaxis_title="Date",
        yaxis_title="Cost (USD)",
        xaxis=dict(tickangle=-45),
        legend_title="Label",
        hovermode="x unified",
        yaxis_tickprefix="$",
        yaxis_tickformat=","
    )

    fig_filtered_line.show()

    display(google.colab.data_table.DataTable(daily_cost_filtered))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        project_name = project_id_input if project_id_input else "all_projects"
        subset_start_date = subset_start_date_input if subset_start_date_input else start_date
        subset_end_date = subset_end_date_input if subset_end_date_input else end_date
        file_name = f"/content/daily_billing_data_for_{project_name}_from_{subset_start_date}_to_{subset_end_date}.xlsx"

        daily_cost_filtered.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)

    display(download_button)

else:
    print("No data found for the selected filters.")


Enter a Project ID to filter (or press Enter to include all): nyt-tafi-prd
Enter subset start date (YYYY-MM-DD) or press Enter to use 2024-10-01: 
Enter subset end date (YYYY-MM-DD) or press Enter to use 2025-03-19: 


,usage_date,label,cost
509,2025-03-19,Storage,2.42
508,2025-03-19,Compute,12.41
507,2025-03-19,BigQuery Others,0.00
506,2025-03-18,Storage,6.75
505,2025-03-18,Compute,44.60
...,...,...,...
3,2024-10-02,BigQuery Others,0.00
5,2024-10-02,Storage,9.40
2,2024-10-01,Storage,9.37
1,2024-10-01,Compute,132.82


Button(description='💾 Download as Excel', style=ButtonStyle())

---
## 📊 Compute Cost  
---

In [86]:
#@title **README**
#@markdown BigQuery offers 2 Compute pricing models:
#@markdown - **Analysis-Based**: Charged by data volume processed.
#@markdown - **Reservation-Based**: GPU-based pricing, where compute resources are allocated in slots, and costs adjust dynamically based on actual usage.
#@markdown
#@markdown The data in this section is an approximation. The analysis-type costs are 99.99% aligned with billing data. However, reservation costs are harder to estimate becasue it's dynamic and less accurate on approximation.
#@markdown
#@markdown 💡 Tip: Switching to the optimal model can reduce costs. Estimated potential savings are provided.
#@markdown Please contact *DBRE* Team (`#bigquery-admin` channel) for pricing change inquiries.




In [85]:
#@title **Compute Part I - Extract Compute Data**
#@markdown 👈 **Click this Play button** to start.



print(f"Fetching detailed Compute Job data from {subset_start_date} to {subset_end_date} for project '{project_id_input if project_id_input else 'All Projects'}'.")

# Define SQL Query dynamically using user inputs
compute_query = f"""
WITH base AS (
  SELECT
    DATE(start_time, 'America/New_York') AS job_start_date,
    user_email,
    CASE
      WHEN reservation_id LIKE '%.%' THEN SPLIT(reservation_id, '.')[OFFSET(1)]
      ELSE reservation_id
    END AS reservation_name,
    project_id,
    job_id,
    job_type,
    statement_type,
    destination_table.project_id AS destination_project_id,
    destination_table.dataset_id AS destination_dataset_id,
    destination_table.table_id AS destination_table_id,
    total_slot_ms,
    total_bytes_billed
  FROM `nyt-platform-analytics-dbt.prod.stg_bq__jobs_by_organization`
  WHERE 1=1
    AND _pt BETWEEN '{subset_start_date}' AND '{subset_end_date}'
    {f"AND project_id = '{project_id_input}'" if project_id_input else ""}
)

SELECT
  job_start_date,
  user_email,
  reservation_name,
  CASE
    WHEN reservation_name IS NULL THEN 'Analysis'
    WHEN reservation_name IS NOT NULL THEN 'Reservation'
  END AS current_pricing_model,
  project_id,
  job_id,
  job_type,
  destination_project_id,
  destination_dataset_id,
  destination_table_id,
  SUM(total_slot_ms/1000/60/60) AS period_slot_hr,
  SUM(total_bytes_billed/POWER(2, 40)) AS total_tb_billed,
  SUM(
    CASE
      WHEN reservation_name IS NULL THEN total_bytes_billed/POWER(2, 40) * 5.1875
      WHEN reservation_name IS NOT NULL THEN total_slot_ms/1000/60/60 * 0.0522
    END
    ) AS cost,
  SUM(
    CASE
      WHEN reservation_name IS NOT NULL THEN total_bytes_billed/POWER(2, 40) * 5.1875
      WHEN reservation_name IS NULL THEN total_slot_ms/1000/60/60 * 0.0522
    END
    ) AS alternative_cost,
  bqutil.fn.job_url(project_id || ':us.' || job_id) AS job_url
FROM base
WHERE 1=1
  AND NOT (reservation_name IS NULL AND statement_type = 'SCRIPT')
GROUP BY ALL
ORDER BY job_start_date DESC
"""
# ANSI Color Code
RED = "\033[31m"
GREEN = "\033[32m"
BOLD = "\033[1m"
RESET = "\033[0m"

try:
    compute_df = client.query(compute_query).to_dataframe()
    print(f"{GREEN}{BOLD} BigQuery Compute Job data from {subset_start_date} to {subset_end_date} for project '{project_id_input if project_id_input else 'All Projects'}' is ready.{RESET}")
except Exception as e:
    print(f"{RED}{BOLD}Query failed: {e}{RESET}")


Fetching detailed Compute Job data from 2024-10-01 to 2025-03-19 for project 'nyt-tafi-prd'.
 BigQuery Compute Job data from 2024-10-01 to 2025-03-19 for project 'nyt-tafi-prd' is ready.


In [59]:
#@title **Compute Part II - User Level Cost & Alternative Cost**
#@markdown 👈 **Click this Play button** to start.



import pandas as pd
import google.colab.data_table

# ANSI Color Code for Red
RED = "\033[31m"
RESET = "\033[0m"  # Reset color to default

# Ensure compute_df has necessary columns
if not compute_df.empty:
    compute_user_df = compute_df.copy()

    compute_user_df["job_start_date"] = pd.to_datetime(compute_user_df["job_start_date"])
    compute_user_df["year_month"] = compute_user_df["job_start_date"].dt.to_period("M").astype(str)

    compute_user_df["potential_savings_from_alternative_pricing"] = \
        compute_user_df["cost"] - compute_user_df["alternative_cost"]

    compute_user_df = compute_user_df.groupby([
        "year_month", "project_id", "current_pricing_model", "user_email"
    ])[["cost", "potential_savings_from_alternative_pricing"]].sum().reset_index()

    compute_user_df[["cost", "potential_savings_from_alternative_pricing"]] = \
        compute_user_df[["cost", "potential_savings_from_alternative_pricing"]].round(2)

    compute_user_df = compute_user_df.sort_values(
        by=["year_month", "potential_savings_from_alternative_pricing"], ascending=[False, False]
    )


    print(f"{RED}⚠️ WARNING: If you see 'potential_savings_from_alternative_pricing' as a negative value, it means you are already on the optimal pricing.{RESET}")
    display(google.colab.data_table.DataTable(compute_user_df, include_index=False))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        project_name = project_id_input if project_id_input else "all_projects"
        file_name = f"/content/high_compute_costs_jobs_for_{project_name}.xlsx"
        compute_user_df.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)
    display(download_button)

else:
    print("No compute job data available for aggregation.")



⚠️ WARNING: If you see 'potential_savings_from_alternative_pricing' as a negative value, it means you are already on the optimal pricing.


,year_month,project_id,current_pricing_model,user_email,cost,potential_savings_from_alternative_pricing
41,2025-03,nyt-tafi-prd,Reservation,facebook-url-expansion@nyt-tafi-prd.iam.gservi...,0.87,0.83
37,2025-03,nyt-tafi-prd,Reservation,108267204623-compute@developer.gserviceaccount...,0.01,0.00
39,2025-03,nyt-tafi-prd,Reservation,anthony.dipietro@nytimes.com,0.00,0.00
40,2025-03,nyt-tafi-prd,Reservation,census-880e72b64ac46d36676dda7@sutrolabs-giza-...,0.00,-0.01
42,2025-03,nyt-tafi-prd,Reservation,james.moon@nytimes.com,0.09,-1.79
38,2025-03,nyt-tafi-prd,Reservation,207932702024-compute@developer.gserviceaccount...,10.58,-96.04
43,2025-03,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,1515.31,-1019.88
33,2025-02,nyt-tafi-prd,Reservation,facebook-url-expansion@nyt-tafi-prd.iam.gservi...,1.54,1.48
29,2025-02,nyt-tafi-prd,Reservation,108267204623-compute@developer.gserviceaccount...,0.01,0.00
31,2025-02,nyt-tafi-prd,Reservation,anthony.dipietro@nytimes.com,0.00,-0.00


Button(description='💾 Download as Excel', style=ButtonStyle())

In [81]:
#@title **Compute Part III - Job Level Cost & Alternative Cost**
#@markdown 👈 **Click this Play button** to start.
#@markdown
#@markdown Enter a `user_email` to explore the top 200 highest-cost jobs per month (≥ $1) and focus on the most impactful queries.
#@markdown
#@markdown Click **💾 Download as Excel** to save the data locally.

import pandas as pd
import google.colab.data_table
import ipywidgets as widgets
from google.colab import files

# ANSI Color Code for Red
RED = "\033[31m"
BOLD = "\033[1m"
RESET = "\033[0m"  # Reset color to default

# User input for user_email filter
user_email_input = input("Enter a user email to filter (or press Enter to include all): ").strip()

# Ensure compute_df has necessary columns
if not compute_df.empty:
    compute_job_df = compute_df.copy()

    compute_job_df["job_start_date"] = pd.to_datetime(compute_job_df["job_start_date"])
    compute_job_df["year_month"] = compute_job_df["job_start_date"].dt.to_period("M").astype(str)

    if user_email_input:
        compute_job_df = compute_job_df.loc[compute_job_df["user_email"] == user_email_input]

    compute_job_df = compute_job_df.copy()
    compute_job_df["potential_savings_from_alternative_pricing"] = \
        compute_job_df["cost"] - compute_job_df["alternative_cost"]

    compute_job_df = compute_job_df.groupby([
        "year_month", "job_start_date", "project_id", "current_pricing_model", "user_email", "job_url", "destination_project_id", "destination_dataset_id", "destination_table_id"
    ])[["cost", "potential_savings_from_alternative_pricing"]].sum().reset_index()

    # Leave NaN values if no data available
    compute_job_df[["cost", "potential_savings_from_alternative_pricing"]] = \
        compute_job_df[["cost", "potential_savings_from_alternative_pricing"]].apply(lambda x: x.where(x.notna(), None))

    compute_job_df[["cost", "potential_savings_from_alternative_pricing"]] = \
        compute_job_df[["cost", "potential_savings_from_alternative_pricing"]].round(2)

    # Filter jobs with cost ≥ 1
    compute_job_df = compute_job_df[compute_job_df["cost"] >= 1]

    compute_job_df = compute_job_df.sort_values(by=["year_month", "cost"], ascending=[False, False])

    compute_job_df = compute_job_df.groupby("year_month").head(200)

    compute_job_df["job_start_date"] = compute_job_df["job_start_date"].dt.strftime('%Y-%m-%d')

    compute_job_df = compute_job_df[
        ["job_start_date", "project_id", "current_pricing_model", "user_email", "cost", "potential_savings_from_alternative_pricing", "job_url", "destination_project_id", "destination_dataset_id", "destination_table_id"]
    ]

    print(f"{RED}⚠️ WARNING: If you see {BOLD}potential_savings_from_alternative_pricing as a negative value{RESET} {RED}, it means you are already on the optimal pricing.{RESET}")
    display(google.colab.data_table.DataTable(compute_job_df, include_index=False))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        user_name = user_email_input if user_email_input else "all_users"
        file_name = f"/content/high_compute_costs_jobs_for_{user_name}.xlsx"
        compute_job_df.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)
    display(download_button)

else:
    print("No compute job data available for aggregation.")


Enter a user email to filter (or press Enter to include all): 
⚠️ WARNING: If you see potential_savings_from_alternative_pricing as a negative value , it means you are already on the optimal pricing.


,job_start_date,project_id,current_pricing_model,user_email,cost,potential_savings_from_alternative_pricing,job_url,destination_project_id,destination_dataset_id,destination_table_id
228781,2025-03-01,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,380.71,-90.86,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,cpo_modeling_prd,evergreen_scoring_dataset
228782,2025-03-01,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,291.50,-207.92,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,cpo_modeling_prd,subrate_intermediate
229056,2025-03-02,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,64.16,32.11,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,_e34af4fc116e8d37efee6d56dd9a0edd3ad4a051,anonc0d6d7d706617222b86fbffcf4e51809283a4b7b65...
231725,2025-03-07,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,34.30,-3.12,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,cpo_cooking,scoring_dataset_saves$20250304
234698,2025-03-14,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,32.41,-5.33,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,cpo_cooking,scoring_dataset_saves$20250311
...,...,...,...,...,...,...,...,...,...,...
21706,2024-10-17,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,1.17,-2.85,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,cpo_modeling_prd,scoring_dataset_pvs$20241016
44078,2024-10-31,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,1.17,-2.95,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,cpo_modeling_prd,scoring_dataset_pvs$20241030
19123,2024-10-15,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,1.16,-0.48,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,_e34af4fc116e8d37efee6d56dd9a0edd3ad4a051,anon7d879558c23bd01358ee76a584b29f2d70de3241ed...
8060,2024-10-06,nyt-tafi-prd,Reservation,nyt-tafi-prd@appspot.gserviceaccount.com,1.15,-0.48,https://console.cloud.google.com/bigquery?proj...,nyt-tafi-prd,_e34af4fc116e8d37efee6d56dd9a0edd3ad4a051,anon578065f308aec1f9e22cfdddd9849bfc2ebf6578e6...


Button(description='💾 Download as Excel', style=ButtonStyle())

In [40]:
#@title **Compute Part IV - Dataset Level Cost**
#@markdown 👈 **Click this Play button** to start.
#@markdown
#@markdown Click **💾 Download as Excel** to save the data locally.

import pandas as pd
import google.colab.data_table
import ipywidgets as widgets
from google.colab import files

# Ensure compute_df has necessary columns
if not compute_df.empty:
    dataset_compute_df = compute_df.copy()

    dataset_compute_df["job_start_date"] = pd.to_datetime(dataset_compute_df["job_start_date"])

    dataset_compute_df["year_month"] = dataset_compute_df["job_start_date"].dt.to_period("M").astype(str)

    dataset_compute_df = dataset_compute_df.groupby(
        ["year_month", "project_id", "destination_project_id", "destination_dataset_id"]
    )["cost"].sum().reset_index()

    dataset_compute_df["cost"] = dataset_compute_df["cost"].round(2)

    dataset_compute_df = dataset_compute_df.sort_values(
        by=["year_month", "cost"], ascending=[False, False]
    )

    display(google.colab.data_table.DataTable(dataset_compute_df, include_index=False))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        project_name = project_id_input if project_id_input else "all_projects"
        file_name = f"/content/high_compute_costs_datasets_for_{project_name}.xlsx"

        dataset_compute_df.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)

    display(download_button)

else:
    print("No compute job data available for aggregation.")


,year_month,project_id,destination_project_id,destination_dataset_id,cost
5,2025-03,nyt-uhub-prd,nyt-uhub-prd,_a86fe362e7cc0921e305481a843d2efa2cf3b466,461.58
1,2025-03,nyt-uhub-prd,nyt-uhub-prd,_03623eb58efce354fd635beb1372152b42d3db17,119.41
2,2025-03,nyt-uhub-prd,nyt-uhub-prd,_1eda2a9032eebf110f6f5a4a6d2e00ca85412089,35.57
3,2025-03,nyt-uhub-prd,nyt-uhub-prd,_8cde9348f919aed73bf5b6dfd6c2a30e8224ebe5,13.36
7,2025-03,nyt-uhub-prd,nyt-uhub-prd,batch_manager_dataset_archive,7.70
8,2025-03,nyt-uhub-prd,nyt-uhub-prd,batch_manager_ots,6.18
6,2025-03,nyt-uhub-prd,nyt-uhub-prd,batch_manager_dataset,2.01
0,2025-03,nyt-uhub-prd,nyt-uhub-prd,_02308522d1432f77b04d2ec2a9f131b399480f4a,1.45
4,2025-03,nyt-uhub-prd,nyt-uhub-prd,_a7021d5a17552ed90d66175571ea0ddb58510379,0.45


Button(description='💾 Download as Excel', style=ButtonStyle())

In [41]:
#@title **Compute Part V - Table Level Cost**
#@markdown 👈 **Click this Play button** to start.
#@markdown
#@markdown Enter a `destination_dataset_id` to explore table-level compute costs, displaying only the top 200 highest-cost tables per dataset per month.
#@markdown
#@markdown Click **💾 Download as Excel** to save the data locally.


import pandas as pd
import google.colab.data_table
import ipywidgets as widgets
from google.colab import files

# User input for destination_table_dataset_id filter
destination_dataset_input = input("Enter a destination dataset (destination_table_dataset_id) to filter (or press Enter to include all): ").strip()

if not compute_df.empty:
    table_compute_df = compute_df.copy()

    table_compute_df["job_start_date"] = pd.to_datetime(table_compute_df["job_start_date"])

    table_compute_df["year_month"] = table_compute_df["job_start_date"].dt.to_period("M").astype(str)

    if destination_dataset_input:
        table_compute_df = table_compute_df[table_compute_df["destination_dataset_id"] == destination_dataset_input]

    table_compute_df = table_compute_df.groupby(
        ["year_month", "project_id", "destination_project_id", "destination_dataset_id", "destination_table_id"]
    )["cost"].sum().reset_index()

    table_compute_df["cost"] = table_compute_df["cost"].round(2)

    table_compute_df = table_compute_df.sort_values(by=["year_month", "cost"], ascending=[False, False])

    # Keep only the **top 200 highest-cost tables per dataset per month**
    table_compute_df = table_compute_df.groupby(["year_month"]).head(200)

    display(google.colab.data_table.DataTable(table_compute_df, include_index=False))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        project_name = project_id_input if project_id_input else "all_projects"
        dataset_name = destination_dataset_input if destination_dataset_input else "all_datasets"
        file_name = f"/content/high_compute_costs_tables_for_{project_name}_{dataset_name}.xlsx"

        table_compute_df.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)

    display(download_button)

else:
    print("No compute job data available for aggregation.")


Enter a destination dataset (destination_table_dataset_id) to filter (or press Enter to include all): 


,year_month,project_id,destination_project_id,destination_dataset_id,destination_table_id,cost
62,2025-03,nyt-uhub-prd,nyt-uhub-prd,_03623eb58efce354fd635beb1372152b42d3db17,anon5190b7d0_69fd_488d_b983_d3ba446fb449,36.33
65,2025-03,nyt-uhub-prd,nyt-uhub-prd,_03623eb58efce354fd635beb1372152b42d3db17,anon6f4c2ef3_30ba_4350_8e5e_24b1a7ca07a0,36.33
61,2025-03,nyt-uhub-prd,nyt-uhub-prd,_03623eb58efce354fd635beb1372152b42d3db17,anon31269975_396e_469f_9d48_a0b3f695a300,7.79
66,2025-03,nyt-uhub-prd,nyt-uhub-prd,_03623eb58efce354fd635beb1372152b42d3db17,anonaff62691_1dcd_43c7_b53f_4a03baec297d,7.79
67,2025-03,nyt-uhub-prd,nyt-uhub-prd,_03623eb58efce354fd635beb1372152b42d3db17,anonbae367c4_061f_4e8d_b1ea_2bc571685568,7.79
...,...,...,...,...,...,...
228,2025-03,nyt-uhub-prd,nyt-uhub-prd,_1eda2a9032eebf110f6f5a4a6d2e00ca85412089,anond3ab56aa_1ea7_413b_b542_d0fc0c1f9743,0.17
229,2025-03,nyt-uhub-prd,nyt-uhub-prd,_1eda2a9032eebf110f6f5a4a6d2e00ca85412089,anond3d56c38_f672_42bb_abf6_82fa35d5ad7d,0.17
230,2025-03,nyt-uhub-prd,nyt-uhub-prd,_1eda2a9032eebf110f6f5a4a6d2e00ca85412089,anond5b06ea8_2741_43ab_8e6f_ff19e690cde0,0.17
231,2025-03,nyt-uhub-prd,nyt-uhub-prd,_1eda2a9032eebf110f6f5a4a6d2e00ca85412089,anond69bb96a_315a_4a5e_84cb_51c6a5e5a408,0.17


Button(description='💾 Download as Excel', style=ButtonStyle())

---
## 🗄️ Storage Cost  
---

In [80]:
#@title **README**
#@markdown BigQuery storage is charged under 2 models:
#@markdown - **Physical Storage**: Default for most tables.
#@markdown - **Logical Storage**: Used by some tables; confirm with the project owner if unclear.
#@markdown
#@markdown Unlike Compute costs, Storage costs accumulate over time:
#@markdown - **Active Storage**: Modified within 90 days.
#@markdown - **Long-Term Storage**: Unused for 90+ days.
#@markdown
#@markdown Note: This analysis focuses only on BASE TABLES for the most accurate results.
#@markdown
#@markdown 💡 Tip: Focus on Long-Term storage savings by deleting unnecessary tables and setting proper retention policies.


In [65]:
#@title **Storage Part I - Extract Storage Data**
#@markdown 👈 **Click this Play button** to start.




# Debugging: Print final date range
print(f"Retrieving BigQuery Table Storage data from {start_date} to {end_date} for project '{project_id_input if project_id_input else 'All Projects'}'.")


# Define SQL query
storage_query = f"""
  SELECT
    DATE_TRUNC(_pt, MONTH) AS month,
    project_id,
    table_schema AS dataset_id,
    table_name AS table_id,
    creation_time,
    table_type,
    ROUND(SUM(active_physical_cost), 2) AS active_physical_cost,
    ROUND(SUM(long_term_physical_cost), 2) AS long_term_physical_cost,
    ROUND(SUM(active_logical_cost), 2) AS active_logical_cost,
    ROUND(SUM(long_term_logical_cost), 2) AS long_term_logical_cost
  FROM `nyt-platform-analytics-dbt.prd_bigquery.int_bq__table_daily_storage_costs`
  WHERE 1=1
    AND _pt BETWEEN '{subset_start_date}' AND '{subset_end_date}'
    {f"AND project_id = '{project_id_input}'" if project_id_input else ""}
  GROUP BY ALL
  ORDER BY long_term_physical_cost DESC
"""

# ANSI Color Code
RED = "\033[31m"
GREEN = "\033[32m"
BOLD = "\033[1m"
RESET = "\033[0m"


try:
    storage_df = client.query(storage_query).to_dataframe()
    print(f"{GREEN}{BOLD}BigQuery Table Storage data from {subset_start_date} to {subset_end_date} for project '{project_id_input if project_id_input else 'All Projects'}' is ready.{RESET}")
except Exception as e:
    print(f"{RED}{BOLD}Query failed: {e}{RESET}")



Retrieving BigQuery Table Storage data from 2024-10-01 to 2025-03-19 for project 'nyt-tafi-prd'.
BigQuery Table Storage data from 2024-10-01 to 2025-03-19 for project 'nyt-tafi-prd' is ready.


In [78]:
#@title **Storage Part II - Dataset Level Cost**
#@markdown 👈 **Click this Play button** to start.
#@markdown
#@markdown Click the **💾 Download as Excel** button below to save the dataset-level storage cost data locally.

import pandas as pd
import google.colab.data_table
import ipywidgets as widgets
from google.colab import files

# ANSI Color Code for Red
RED = "\033[31m"
ITALIC = "\033[3m"
RESET = "\033[0m"  # Reset color to default

# Ensure storage_df has necessary columns and apply filters
if not storage_df.empty:
    dataset_level_df = storage_df.copy()

    dataset_level_df["month"] = pd.to_datetime(dataset_level_df["month"])

    # Filter only BASE TABLES
    dataset_level_df = dataset_level_df[dataset_level_df["table_type"] == "BASE TABLE"]

    aggregated_dataset_df = dataset_level_df.groupby(
        ["month", "project_id", "dataset_id"]
    )[
        ["active_physical_cost", "long_term_physical_cost", "active_logical_cost", "long_term_logical_cost"]
    ].sum().reset_index()

    aggregated_dataset_df[["active_physical_cost", "long_term_physical_cost", "active_logical_cost", "long_term_logical_cost"]] = \
        aggregated_dataset_df[["active_physical_cost", "long_term_physical_cost", "active_logical_cost", "long_term_logical_cost"]].round(2)

    aggregated_dataset_df = aggregated_dataset_df.sort_values(
        by=["month", "long_term_physical_cost", "dataset_id"], ascending=[False, False, False]
    )

    aggregated_dataset_df["month"] = aggregated_dataset_df["month"].dt.strftime('%Y-%m')


    print(f"{RED}⚠️ This section is still an experimental feature in development.\n"
          f"If you're unsure about the storage type (Physical or Logical) of a dataset,\n"
          f"please reach out to the {ITALIC}Project Owner, DBRE, or Tian{RESET} {RED}for assistance.{RESET}")


    display(google.colab.data_table.DataTable(aggregated_dataset_df, include_index=False))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        project_name = project_id_input if project_id_input else "all_projects"
        file_name = f"/content/high_storage_costs_datasets_for_{project_name}.xlsx"

        aggregated_dataset_df.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)

    display(download_button)

else:
    print("No storage data available for the selected range.")


⚠️ This section is still an experimental feature in development.
If you're unsure about the storage type (Physical or Logical) of a table,
please reach out to the Project Owner, DBRE, or Tian for assistance.


,month,project_id,dataset_id,active_physical_cost,long_term_physical_cost,active_logical_cost,long_term_logical_cost
71,2025-03,nyt-tafi-prd,evergreen,55.29,34.64,69.66,80.27
68,2025-03,nyt-tafi-prd,cpo_modeling_prd,29.63,1.55,30.81,7.21
69,2025-03,nyt-tafi-prd,curious_audience,0.42,0.21,0.89,0.36
77,2025-03,nyt-tafi-prd,tempo,20.53,0.12,10.28,0.17
66,2025-03,nyt-tafi-prd,blossom,0.01,0.09,0.01,0.17
...,...,...,...,...,...,...,...
9,2024-10,nyt-tafi-prd,tafi_gae_log_sink_debug,0.00,0.00,0.00,0.00
8,2024-10,nyt-tafi-prd,stories,19.05,0.00,21.20,0.00
7,2024-10,nyt-tafi-prd,periodic_cooking_algo,0.00,0.00,0.01,0.00
6,2024-10,nyt-tafi-prd,facebook,0.08,0.00,0.20,0.00


Button(description='💾 Download as Excel', style=ButtonStyle())

In [79]:
#@title **Storage Part 3 - Table Level Cost**
#@markdown 👈 **Click this Play button** to start.
#@markdown
#@markdown Enter a `dataset_id_` to filter and explore storage costs at the **table level**.
#@markdown
#@markdown Click the **💾 Download as Excel** button below the table to save the data locally.

import pandas as pd
import google.colab.data_table
import ipywidgets as widgets
from google.colab import files

# ANSI Color Code for Red
RED = "\033[31m"
ITALIC = "\033[3m"
RESET = "\033[0m"  # Reset color to default

# User input for dataset_id filter (moved outside the function)
dataset_id_input = input("Enter a dataset (dataset_id) to filter (or press Enter to include all): ").strip()

# Ensure dataset_level_df has necessary columns
if not dataset_level_df.empty:
    table_level_df = storage_df.copy()

    table_level_df["month"] = pd.to_datetime(table_level_df["month"])

    # Filter only BASE TABLES
    table_level_df = table_level_df[table_level_df["table_type"] == "BASE TABLE"]

    if dataset_id_input:
        table_level_df = table_level_df[table_level_df["dataset_id"] == dataset_id_input]

    table_level_df = table_level_df.groupby(
        ["month", "project_id", "dataset_id", "table_id"]
    )[
        ["active_physical_cost", "long_term_physical_cost", "active_logical_cost", "long_term_logical_cost"]
    ].sum().reset_index()

    table_level_df[["active_physical_cost", "long_term_physical_cost", "active_logical_cost", "long_term_logical_cost"]] = \
        table_level_df[["active_physical_cost", "long_term_physical_cost", "active_logical_cost", "long_term_logical_cost"]].round(2)

    table_level_df = table_level_df.sort_values(
        by=["month", "long_term_physical_cost"], ascending=[False, False]
    )

    table_level_df["month"] = table_level_df["month"].dt.strftime('%Y-%m')

    print(f"{RED}⚠️ This section is still an experimental feature in development.\n"
          f"If you're unsure about the storage type (Physical or Logical) of a table,\n"
          f"please reach out to the {ITALIC}Project Owner, DBRE, or Tian{RESET} {RED}for assistance.{RESET}")

    display(google.colab.data_table.DataTable(table_level_df, include_index=False))

    # --- Download Button ---
    def download_file(change):
        """ Function to handle file download when button is clicked """
        project_name = project_id_input if project_id_input else "all_projects"
        dataset_name = dataset_id_input if dataset_id_input else "all_datasets"
        file_name = f"/content/high_storage_costs_tables_for_{project_name}_{dataset_name}.xlsx"

        table_level_df.to_excel(file_name, index=False)
        print(f"\n✅ Excel file saved successfully: {file_name}")
        files.download(file_name)

    download_button = widgets.Button(description="💾 Download as Excel")
    download_button.on_click(download_file)

    display(download_button)

else:
    print("No storage data available for the selected range.")


Enter a dataset (dataset_id) to filter (or press Enter to include all): evergreen
⚠️ This section is still an experimental feature in development.
If you're unsure about the storage type (Physical or Logical) of a table,
please reach out to the Project Owner, DBRE, or Tian for assistance.


,month,project_id,dataset_id,table_id,active_physical_cost,long_term_physical_cost,active_logical_cost,long_term_logical_cost
140,2025-03,nyt-tafi-prd,evergreen,periodically_evergreen_pageviews_2023,0.00,21.03,0.00,28.94
139,2025-03,nyt-tafi-prd,evergreen,periodically_evergreen_pageviews,0.00,13.36,0.00,50.86
148,2025-03,nyt-tafi-prd,evergreen,timeless_sum_pageviews,0.00,0.16,0.00,0.28
149,2025-03,nyt-tafi-prd,evergreen,timeless_sum_pageviews_pt,0.02,0.06,0.03,0.11
129,2025-03,nyt-tafi-prd,evergreen,daily_cedar_training_dataset,0.00,0.01,0.00,0.02
...,...,...,...,...,...,...,...,...
18,2024-10,nyt-tafi-prd,evergreen,recent_article_scores_temp,0.00,0.00,0.00,0.00
19,2024-10,nyt-tafi-prd,evergreen,scoring_dataset,0.00,0.00,0.00,0.00
20,2024-10,nyt-tafi-prd,evergreen,scoring_pageviews,8.89,0.00,5.04,0.00
21,2024-10,nyt-tafi-prd,evergreen,stela_promotions,0.01,0.00,0.00,0.00


Button(description='💾 Download as Excel', style=ButtonStyle())